In [ ]:
#!/usr/bin/env python3

import os
import glob
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from datetime import datetime
import gc
import seaborn as sns

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

KAGGLE_INPUT_DIR = "/kaggle/input"
KAGGLE_WORKING_DIR = "/kaggle/working"
OUTPUT_DIR = KAGGLE_WORKING_DIR
os.makedirs(OUTPUT_DIR, exist_ok=True)

import sys
sys.stdout.flush()

np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
    torch.backends.cudnn.benchmark = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def load_beijing_data(data_dir="/kaggle/input/beijing-multi-site-air-quality-data"):
    print(f"Looking for data in: {data_dir}")
    all_files = glob.glob(os.path.join(data_dir, "*.csv"))
    
    if not all_files:
        all_files = glob.glob(os.path.join(data_dir, "**/*.csv"), recursive=True)
        
    print(f"Found {len(all_files)} CSV files")
    if len(all_files) == 0:
        data_dir = "data/beijing"
        all_files = glob.glob(os.path.join(data_dir, "*.csv"))
        if not all_files:
            raise FileNotFoundError(f"No CSV files found in {data_dir}")
    
    dfs = []
    
    for filename in tqdm(all_files, desc="Loading CSV files"):
        df = pd.read_csv(filename)
        station = os.path.basename(filename).split('.')[0]
        df['station'] = station
        dfs.append(df)
    
    combined_df = pd.concat(dfs, ignore_index=True)
    
    if 'date' in combined_df.columns and 'time' in combined_df.columns:
        combined_df['datetime'] = pd.to_datetime(combined_df['date'] + ' ' + combined_df['time'])
    elif 'year' in combined_df.columns and 'month' in combined_df.columns and 'day' in combined_df.columns:
        combined_df['datetime'] = pd.to_datetime(combined_df[['year', 'month', 'day', 'hour']])
    
    combined_df.set_index('datetime', inplace=True)
    
    feature_columns = ['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3']
    feature_columns = [col for col in feature_columns if col in combined_df.columns]
    
    print(f"Available features: {feature_columns}")
    print(f"Data shape before cleaning: {combined_df.shape}")
    print("Missing values before cleaning:")
    print(combined_df[feature_columns].isna().sum())
    
    for col in feature_columns:
        combined_df[col] = pd.to_numeric(combined_df[col], errors='coerce')
    
    print("Processing missing values...")
    combined_df[feature_columns] = combined_df.groupby('station')[feature_columns].apply(
        lambda x: x.interpolate(method='linear')
    )
    combined_df = combined_df.fillna(method='bfill').fillna(method='ffill')
    
    print("Missing values after cleaning:")
    print(combined_df[feature_columns].isna().sum())
    
    print("Adding time features...")
    combined_df['hour'] = combined_df.index.hour
    combined_df['day'] = combined_df.index.day
    combined_df['month'] = combined_df.index.month
    combined_df['year'] = combined_df.index.year
    combined_df['dayofweek'] = combined_df.index.dayofweek
    
    print("Basic statistics for key features:")
    print(combined_df[feature_columns].describe())
    
    return combined_df, feature_columns

try:
    df, feature_columns = load_beijing_data()
    print(f"Data loaded successfully. Shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}")
    print(f"Using features: {feature_columns}")
except Exception as e:
    print(f"Error loading data: {e}")
    print("Creating sample data...")
    dates = pd.date_range(start='2013-01-01', end='2017-12-31', freq='H')
    stations = ['Aotizhongxin', 'Changping', 'Dingling', 'Dongsi', 'Guanyuan', 'Gucheng']
    
    sample_data = []
    for station in tqdm(stations, desc="Creating sample data"):
        station_data = pd.DataFrame(index=dates)
        station_data['PM2.5'] = np.random.normal(50, 25, len(dates))
        station_data['PM10'] = np.random.normal(80, 30, len(dates))
        station_data['SO2'] = np.random.normal(20, 10, len(dates))
        station_data['NO2'] = np.random.normal(40, 15, len(dates))
        station_data['CO'] = np.random.normal(1, 0.5, len(dates))
        station_data['O3'] = np.random.normal(60, 20, len(dates))
        station_data['station'] = station
        
        station_data['hour'] = station_data.index.hour
        station_data['day'] = station_data.index.day
        station_data['month'] = station_data.index.month
        station_data['year'] = station_data.index.year
        station_data['dayofweek'] = station_data.index.dayofweek
        
        sample_data.append(station_data)
    
    df = pd.concat(sample_data)
    feature_columns = ['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3']
    print(f"Sample data created. Shape: {df.shape}")

for i, feature in enumerate(feature_columns):
    plt.subplot(len(feature_columns), 1, i+1)
    sample_station = df['station'].unique()[0]
    sample_data = df[df['station'] == sample_station].iloc[-720:][feature]
    plt.plot(sample_data.index, sample_data.values)
    plt.title(f'{feature} - Station: {sample_station}')
    plt.grid(True)
plt.tight_layout()
plt.show()

print("One-hot encoding categorical variables...")
df_encoded = pd.get_dummies(df, columns=['station'])
print(f"Shape after one-hot encoding: {df_encoded.shape}")

print("Scaling features...")
cols_to_scale = df_encoded.columns.tolist()
scaler = StandardScaler()
df_encoded_scaled = pd.DataFrame(
    scaler.fit_transform(df_encoded),
    index=df_encoded.index,
    columns=df_encoded.columns
)

sampled_cols = df_encoded_scaled.columns[:5].tolist()
print(f"Mean: {df_encoded_scaled[sampled_cols].mean().to_dict()}")
print(f"Std: {df_encoded_scaled[sampled_cols].std().to_dict()}")

context_length = 168
prediction_length = 24
target_cols = feature_columns

print(f"Context length: {context_length} hours")
print(f"Prediction length: {prediction_length} hours")
print(f"Target columns: {target_cols}")

print("Splitting data...")
train_cutoff = pd.Timestamp('2016-12-31')
val_cutoff = pd.Timestamp('2017-06-30')

train_data = df_encoded_scaled[df_encoded_scaled.index <= train_cutoff]
val_data = df_encoded_scaled[(df_encoded_scaled.index > train_cutoff) & (df_encoded_scaled.index <= val_cutoff)]
test_data = df_encoded_scaled[df_encoded_scaled.index > val_cutoff]

print(f"Train data shape: {train_data.shape}, from {train_data.index.min()} to {train_data.index.max()}")
print(f"Validation data shape: {val_data.shape}, from {val_data.index.min()} to {val_data.index.max()}")
print(f"Test data shape: {test_data.shape}, from {test_data.index.min()} to {test_data.index.max()}")

class TimeSeriesDataset(Dataset):
    def __init__(self, data, context_length, prediction_length, target_cols):
        self.data = data
        self.context_length = context_length
        self.prediction_length = prediction_length
        self.target_indices = [data.columns.get_loc(col) for col in target_cols]
        self.total_length = context_length + prediction_length
        
    def __len__(self):
        return max(0, len(self.data) - self.total_length + 1)
    
    def __getitem__(self, idx):
        x_start = idx
        x_end = idx + self.context_length
        x = self.data.iloc[x_start:x_end].values
        
        y_start = x_end
        y_end = y_start + self.prediction_length
        y = self.data.iloc[y_start:y_end, self.target_indices].values
        
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

print("Creating datasets...")
train_dataset = TimeSeriesDataset(train_data, context_length, prediction_length, target_cols)
val_dataset = TimeSeriesDataset(val_data, context_length, prediction_length, target_cols)
test_dataset = TimeSeriesDataset(test_data, context_length, prediction_length, target_cols)

print(f"Dataset sizes - Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}")

batch_size = 128
num_workers = 0 if device.type == 'cpu' else 2

print("Creating dataloaders...")
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, 
                         num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, 
                       num_workers=num_workers, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, 
                        num_workers=num_workers, pin_memory=True)

print(f"Created dataloaders - Train: {len(train_loader)}, Val: {len(val_loader)}, Test: {len(test_loader)} batches")

try:
    from timesfm.torch import TimesFM
    
    class TimesFMWithFC(nn.Module):
        def __init__(self, timesfm_model, input_dim, hidden_dim=256, num_targets=None):
            super().__init__()
            self.timesfm = timesfm_model
            self.fc1 = nn.Linear(input_dim, hidden_dim)
            self.dropout = nn.Dropout(0.2)
            self.layer_norm = nn.LayerNorm(hidden_dim)
            self.relu = nn.ReLU()
            self.fc2 = nn.Linear(hidden_dim, num_targets)
            self.num_targets = num_targets
            
        def forward(self, x, prediction_length, temperature=1.0):
            timesfm_output = self.timesfm(
                x,
                prediction_length=prediction_length,
                temperature=temperature
            )
            
            batch_size = timesfm_output.size(0)
            reshaped = timesfm_output.reshape(-1, timesfm_output.size(-1))
            
            fc1_out = self.fc1(reshaped)
            fc1_out = self.layer_norm(fc1_out)
            fc1_out = self.relu(fc1_out)
            fc1_out = self.dropout(fc1_out)
            fc2_out = self.fc2(fc1_out)
            
            output = fc2_out.reshape(batch_size, prediction_length, self.num_targets)
            
            return output
    
    print("Loading TimesFM pre-trained model...")
    with tqdm(total=1, desc="Loading pre-trained model") as pbar:
        base_model = TimesFM.from_pretrained("google-research/timesfm-1.0-200m", use_cache=True)
        pbar.update(1)
    model = TimesFMWithFC(base_model, df_encoded.shape[1], hidden_dim=256, num_targets=len(target_cols))
    model.to(device)
    print("TimesFM model with FC layers loaded successfully")
    model_type = "TimesFM + FC"
    
except Exception as e:
    print(f"Error loading TimesFM model: {e}")
    print("Creating enhanced LSTM model with FC layers")
    
    class EnhancedLSTM(nn.Module):
        def __init__(self, input_dim, hidden_dim=256, num_targets=None, pred_len=24):
            super().__init__()
            self.input_embedding = nn.Linear(input_dim, hidden_dim)
            
            self.lstm = nn.LSTM(
                input_size=hidden_dim,
                hidden_size=hidden_dim,
                num_layers=2,
                batch_first=True,
                dropout=0.1,
                bidirectional=True
            )
            
            self.fc1 = nn.Linear(hidden_dim * 2, hidden_dim)
            self.layer_norm = nn.LayerNorm(hidden_dim)
            self.dropout = nn.Dropout(0.2)
            self.relu = nn.ReLU()
            self.fc2 = nn.Linear(hidden_dim, num_targets)
            
            self.pred_len = pred_len
            self.num_targets = num_targets
            
        def forward(self, x, prediction_length=None, temperature=None):
            batch_size = x.shape[0]
            x = self.input_embedding(x)
            
            x, _ = self.lstm(x)
            
            x = x[:, -1, :]
            
            x = self.fc1(x)
            x = self.layer_norm(x)
            x = self.relu(x)
            x = self.dropout(x)
            x = self.fc2(x)
            
            pred_len = prediction_length or self.pred_len
            
            output = x.unsqueeze(1).repeat(1, pred_len, 1)
            
            return output
    
    with tqdm(total=1, desc="Creating LSTM model") as pbar:
        model = EnhancedLSTM(df_encoded.shape[1], hidden_dim=256, num_targets=len(target_cols), pred_len=prediction_length)
        model.to(device)
        pbar.update(1)
    print("Enhanced LSTM model created")
    model_type = "Enhanced LSTM"

model_summary = f"Model: {model_type} using {len(feature_columns)} features with prediction length {prediction_length}"
print("\n" + "="*80)
print(model_summary)
print("="*80 + "\n")

optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-5)
criterion = nn.MSELoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

num_epochs = 20
best_val_loss = float('inf')
train_losses = []
val_losses = []

print("\nStarting training...")
for epoch in range(num_epochs):
    model.train()
    epoch_train_loss = 0
    train_iter = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]", leave=False)
    
    for batch_idx, (x_batch, y_batch) in enumerate(train_iter):
        x_batch, y_batch = x_batch.to(device, non_blocking=True), y_batch.to(device, non_blocking=True)
        
        optimizer.zero_grad(set_to_none=True)
        
        predictions = model(
            x_batch,
            prediction_length=prediction_length,
            temperature=1.0
        )
        
        loss = criterion(predictions, y_batch)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        epoch_train_loss += loss.item()
        train_iter.set_postfix({"batch_loss": f"{loss.item():.4f}"})
        
        if batch_idx % 10 == 0:
            del x_batch, y_batch, predictions
            if device.type == 'cuda':
                torch.cuda.empty_cache()
    
    avg_train_loss = epoch_train_loss / len(train_loader)
    train_losses.append(avg_train_loss)
    
    model.eval()
    epoch_val_loss = 0
    val_iter = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Valid]", leave=False)
    
    with torch.no_grad():
        for x_batch, y_batch in val_iter:
            x_batch, y_batch = x_batch.to(device, non_blocking=True), y_batch.to(device, non_blocking=True)
            
            predictions = model(
                x_batch,
                prediction_length=prediction_length,
                temperature=1.0
            )
            
            loss = criterion(predictions, y_batch)
            epoch_val_loss += loss.item()
            val_iter.set_postfix({"batch_loss": f"{loss.item():.4f}"})
            
            del x_batch, y_batch, predictions
            if device.type == 'cuda':
                torch.cuda.empty_cache()
    
    avg_val_loss = epoch_val_loss / len(val_loader)
    val_losses.append(avg_val_loss)
    
    scheduler.step(avg_val_loss)
    current_lr = optimizer.param_groups[0]['lr']
    
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, LR: {current_lr:.6f}")
    
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        model_save_path = os.path.join(OUTPUT_DIR, "timesfm_beijing_best_model.pth")
        torch.save(model.state_dict(), model_save_path)
        print(f"✓ Saved best model with validation loss: {best_val_loss:.4f}")
    
    if (epoch + 1) % 5 == 0 or epoch == num_epochs - 1:
        plt.figure()
        plt.plot(range(1, epoch + 2), train_losses, 'b-', label='Training Loss')
        plt.plot(range(1, epoch + 2), val_losses, 'r-', label='Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title(f'Training and Validation Loss - {model_type} (Epoch {epoch+1})')
        plt.legend()
        plt.grid(True)
        plt.show()
    
    gc.collect()
    if device.type == 'cuda':
        torch.cuda.empty_cache()

plt.figure()
plt.plot(range(1, num_epochs+1), train_losses, 'b-', label='Training Loss')
plt.plot(range(1, num_epochs+1), val_losses, 'r-', label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title(f'Training and Validation Loss - {model_type}')
plt.legend()
plt.grid(True)
plt.show()

print("\nLoading best model for evaluation...")
model.load_state_dict(torch.load(os.path.join(OUTPUT_DIR, "timesfm_beijing_best_model.pth")))
model.eval()

test_loss = 0
all_preds = []
all_targets = []

print("\nEvaluating on test set...")
with torch.no_grad():
    for x_batch, y_batch in tqdm(test_loader, desc="Testing"):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        
        predictions = model(
            x_batch,
            prediction_length=prediction_length,
            temperature=1.0
        )
        
        loss = criterion(predictions, y_batch)
        test_loss += loss.item()
        
        all_preds.append(predictions.cpu().numpy())
        all_targets.append(y_batch.cpu().numpy())
        
        del x_batch, y_batch, predictions
        if device.type == 'cuda':
            torch.cuda.empty_cache()

test_loss /= len(test_loader)
print(f"\nTest Loss: {test_loss:.4f}")

if len(all_preds) > 0:
    all_preds = np.concatenate(all_preds, axis=0)
    all_targets = np.concatenate(all_targets, axis=0)
    
    print(f"Prediction shape: {all_preds.shape}, Target shape: {all_targets.shape}")
    
    print("Calculating metrics...")
    metrics_per_hour = []
    for hour in tqdm(range(prediction_length), desc="Calculating hourly metrics"):
        hour_metrics = []
        for i, pollutant in enumerate(target_cols):
            mae = mean_absolute_error(all_targets[:, hour, i], all_preds[:, hour, i])
            rmse = np.sqrt(mean_squared_error(all_targets[:, hour, i], all_preds[:, hour, i]))
            r2 = r2_score(all_targets[:, hour, i], all_preds[:, hour, i])
            hour_metrics.append({
                'Hour': hour + 1,
                'Pollutant': pollutant,
                'MAE': mae,
                'RMSE': rmse,
                'R2': r2
            })
        metrics_per_hour.extend(hour_metrics)
    
    hourly_metrics_df = pd.DataFrame(metrics_per_hour)
    
    print("\nHourly Prediction Metrics (first 12 rows):")
    print(hourly_metrics_df.head(12))
    
    print("Generating plots...")
    pivot_df = hourly_metrics_df.pivot_table(index='Hour', columns='Pollutant', values='RMSE')
    plt.figure()
    for pollutant in pivot_df.columns:
        plt.plot(pivot_df.index, pivot_df[pollutant], marker='o', label=pollutant)
    plt.xlabel('Prediction Hour')
    plt.ylabel('RMSE')
    plt.title('RMSE by Hour for Each Pollutant')
    plt.legend()
    plt.grid(True)
    plt.xticks(range(1, prediction_length + 1))
    plt.show()
    
    pivot_df_r2 = hourly_metrics_df.pivot_table(index='Hour', columns='Pollutant', values='R2')
    plt.figure()
    for pollutant in pivot_df_r2.columns:
        plt.plot(pivot_df_r2.index, pivot_df_r2[pollutant], marker='o', label=pollutant)
    plt.xlabel('Prediction Hour')
    plt.ylabel('R² Score')
    plt.title('R² Score by Hour for Each Pollutant')
    plt.legend()
    plt.grid(True)
    plt.xticks(range(1, prediction_length + 1))
    plt.show()
    
    print("Plotting sample predictions...")
    num_samples = min(5, len(all_preds))
    fig, axes = plt.subplots(len(target_cols), num_samples, figsize=(20, 15), sharex=True)
    
    for i, pollutant in enumerate(target_cols):
        sample_indices = np.random.choice(len(all_preds), num_samples, replace=False)
        
        for j, idx in enumerate(sample_indices):
            ax = axes[i, j]
            ax.plot(range(prediction_length), all_preds[idx, :, i], 'r-', linewidth=2, label='Predicted')
            ax.plot(range(prediction_length), all_targets[idx, :, i], 'b-', linewidth=2, label='Actual')
            ax.set_title(f'{pollutant} - Sample {j+1}')
            ax.grid(True)
            
            if i == len(target_cols) - 1:
                ax.set_xlabel('Hours Ahead')
            
            if j == 0:
                ax.set_ylabel(f'{pollutant} Level')
                
            if i == 0 and j == 0:
                ax.legend()
    
    plt.tight_layout()
    plt.show()
    
    print("Calculating overall metrics...")
    overall_metrics = []
    for i, pollutant in enumerate(tqdm(target_cols, desc="Calculating overall metrics")):
        mae = mean_absolute_error(all_targets[:, :, i].flatten(), all_preds[:, :, i].flatten())
        rmse = np.sqrt(mean_squared_error(all_targets[:, :, i].flatten(), all_preds[:, :, i].flatten()))
        r2 = r2_score(all_targets[:, :, i].flatten(), all_preds[:, :, i].flatten())
        
        overall_metrics.append({
            'Pollutant': pollutant,
            'MAE': mae,
            'RMSE': rmse,
            'R2': r2
        })
    
    metrics_df = pd.DataFrame(overall_metrics)
    
    print("\nOverall Metrics by Pollutant:")
    print(metrics_df)
    
    plt.figure()
    metrics_df.plot(x='Pollutant', y=['MAE', 'RMSE'], kind='bar', ax=plt.gca())
    plt.title('MAE and RMSE by Pollutant')
    plt.ylabel('Error Value')
    plt.grid(axis='y')
    plt.show()
    
    plt.figure()
    plt.bar(metrics_df['Pollutant'], metrics_df['R2'])
    plt.title('R² Score by Pollutant')
    plt.ylabel('R² Score')
    plt.ylim(0, 1)
    plt.grid(axis='y')
    plt.show()
    
    print("Generating heatmaps...")
    rmse_pivot = hourly_metrics_df.pivot_table(index='Hour', columns='Pollutant', values='RMSE')
    plt.figure(figsize=(15, 10))
    ax = sns.heatmap(rmse_pivot, annot=True, cmap='YlOrRd', fmt='.2f')
    plt.title('RMSE by Hour and Pollutant')
    plt.tight_layout()
    plt.show()
    
    r2_pivot = hourly_metrics_df.pivot_table(index='Hour', columns='Pollutant', values='R2')
    plt.figure(figsize=(15, 10))
    ax = sns.heatmap(r2_pivot, annot=True, cmap='viridis', fmt='.2f')
    plt.title('R² Score by Hour and Pollutant')
    plt.tight_layout()
    plt.show()
else:
    print("No predictions were made. Test dataset may be empty or invalid.")

print("\n" + "="*80)
print("Training and evaluation completed successfully!")
print("="*80)